In [1]:
from ipywidgets import interactive, interact
import ipywidgets as widgets
from IPython.display import SVG, display
from matplotlib import colormaps

import json
import math
import chess

In [2]:
cmap = colormaps["PuRd"]

def hexify(f):
    assert f >= 0 and f <= 1
    return f"{int(f * 255):02x}"

def get_color(val):
    if val < 1e-3:
        return f"#ffffff77"
    r, g, b, _ = cmap(val)
    return f"#{hexify(r)}{hexify(g)}{hexify(b)}ff"

def replay(trace, step):
    steps = trace["steps"]
    b = chess.Board()
    for move in steps[0:step]:
        b.push(chess.Move.from_uci(move[0]))

    moves, num_acts, q_values = zip(*steps[step][2])
    num_acts = [n for _, n, _ in steps[step][2]]
    q_values = [v for _, _, v in steps[step][2]]
    sum_num = sum(num_acts) + 1e-4
    score = [v / sum_num for v in num_acts]
    return b, list(zip(moves, score, q_values, num_acts))

In [25]:
filename = "../runs/18/b0/trace3.json"
# filename = "../debug-traces/trace4.json"
# filename = "../trace6.json"
# filename = "../replay.json"

In [26]:
with open(filename) as f:
    trace = json.load(f)

outcome = trace["outcome"]
steps = trace["steps"]
    
def f(step=widgets.IntSlider(min=0, max=len(steps), step=1, value=0)):
    b = chess.Board()
    for move in steps[0:step]:
        b.push(chess.Move.from_uci(move[0]))

    if step == len(steps):
        return b

    moves = [m for m, _, _ in steps[step][2]]
    num_acts = [n for _, n, _ in steps[step][2]]
    sum_num = sum(num_acts) + 1e-4
    score = [v / sum_num for v in num_acts]
    distr = sorted(zip(moves, score), key=lambda p: p[1], reverse=True)
    labels = widgets.HBox([
        widgets.Label(value=f"{m}", style=dict(background=get_color(c)))
        for m, c in distr
    ])
    out = widgets.Output()
    with out:
        display(SVG(data=b._repr_svg_()))
    return widgets.VBox([out, labels])

interact(f);

interactive(children=(IntSlider(value=0, description='step', max=128), Output()), _dom_classes=('widget-intera…

In [122]:
outcome

{'termination': 'Checkmate', 'winner': 'Black'}

In [12]:
from importlib import reload
import train
import libencoder
reload(train)
reload(libencoder)

<module 'libencoder' from '/home2/jiasen/workspace/smart-chess-rust/target/release/libencoder.so'>

In [14]:
import torch
import nn
model = nn.load_model(
    device="cpu", 
    checkpoint="../runs/17/tb_logs/chess/version_3/epoch-9.ckpt",
    compile=False,
)

..loading checkpoint:  ../runs/17/tb_logs/chess/version_3/epoch-9.ckpt


/home2/jiasen/workspace/smart-chess-rust/venv/lib/python3.11/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [15]:
def infer(fn, index):
    ds = train.ChessDataset(fn)

    board_enc = ds[index][0]
    moves_enc = ds.steps[index][3]
  
    pi, value = model(board_enc.unsqueeze(0))
    pi = torch.exp(pi.detach()).squeeze()

    pi = pi[moves_enc]
    pi = pi / pi.sum()

    value = value.detach().squeeze()
    return pi, value

In [16]:
with open(filename) as f:
    trace = json.load(f)

step = 9
prior, value = infer(filename, step)
total_n = sum([n for _, n, _ in trace["steps"][step][2]])

distr = replay(trace, step)

In [17]:
total_n, prior, value

(357,
 tensor([0.0921, 0.0336, 0.0658, 0.0470, 0.1290, 0.0240, 0.0123, 0.0336, 0.0240,
         0.0336, 0.0172, 0.0240, 0.0470, 0.0336, 0.0470, 0.0240, 0.1290, 0.0088,
         0.0240, 0.0336, 0.0172, 0.0336, 0.0658]),
 tensor(0.0234))

In [18]:
def uct(prior, total_n, q_value, current_n, reverse, cpuct):
    award = q_value / (current_n + 1e-5) * (-1 if reverse else 1)
    exploration = math.sqrt(total_n) / (1 + current_n) * prior * cpuct
    return (f"{award + exploration:0.3f}", f"{award:0.3f}", f"{exploration:0.3f}")

In [19]:
uct(0.052366, 160, -0.043745, 1, False, 0.2)

('0.022', '-0.044', '0.066')

In [20]:
uct(0.040944, 160, 0.087995, 1, False, 0.2)

('0.140', '0.088', '0.052')

In [24]:
rotate = step % 2 == 1
cpuct = 2
import pandas as pd
pd.DataFrame(
    [(m, n, q, prior[i].item(), uct(prior[i].item(), total_n, q, n, rotate, cpuct)) for i, (m, n, q) in enumerate(trace["steps"][step][2])],
    columns=("move", "nact", "q", "prior", "uct"),
)

,move,nact,q,prior,uct
0,h8h7,11,-0.803467,0.092141,"(0.363, 0.073, 0.290)"
1,g8f6,10,-0.169168,0.033615,"(0.132, 0.017, 0.115)"
2,d8f8,7,-0.046951,0.065839,"(0.318, 0.007, 0.311)"
3,d8e8,10,-0.546410,0.047045,"(0.216, 0.055, 0.162)"
4,d8c7,9,-0.779129,0.128951,"(0.574, 0.087, 0.487)"
5,d8b6,8,0.042175,0.024020,"(0.096, -0.005, 0.101)"
6,d8a5,9,0.134937,0.012264,"(0.031, -0.015, 0.046)"
7,b8c6,10,-0.331546,0.033615,"(0.149, 0.033, 0.115)"
8,b8a6,10,-0.113235,0.024020,"(0.094, 0.011, 0.083)"
9,d7c7,42,-6.221313,0.033615,"(0.178, 0.148, 0.030)"


In [13]:
data = [(chess.Move.from_uci(step[0]), [n[1]for n in step[2]]) for step in trace["steps"]]

# step16 = [0] * 22
# step16[4] = 1
# ret = libencoder.encode(data[:16] + [(chess.Move.from_uci("b5c3"), step16)])
ret = libencoder.encode(data)
eb, em, ed, mi = ret[22]

inp = np.concatenate((eb, em), axis=-1).astype(np.float32)
inp = inp.transpose((2, 0, 1))

inp = torch.from_numpy(inp).unsqueeze(0)
pi, value = model(inp)
pi = torch.exp(pi.detach()).squeeze()
pi = pi[mi]
pi = pi / pi.sum()
value = value.detach().squeeze()
pi, value

NameError: name 'np' is not defined

In [9]:
data = [(chess.Move.from_uci(step[0]), [n[1]for n in step[2]]) for step in trace["steps"]]
ret = libencoder.encode_steps(data)
eb, em, ed, mi = ret[35]
em[0]

RS 1 0 0 [1 0] 0
RS 1 7 0
RS 1 0 0 [1 0] 0
RS 1 6 0
RS 1 0 0 [1 0] 0
RS 1 5 0
RS 1 0 0 [1 0] 0
RS 1 4 0
RS 1 0 0 [1 0] 0
RS 1 3 0
RS 1 0 0 [1 0] 0
RS 1 2 0
RS 1 0 0 [1 0] 0
RS 1 1 0
RS 1 0 0 [1 0] 0
RS 1 0 0
RS 2 0 1 [1 0] 0
RS 1 7 1
RS 2 0 1 [1 0] 0
RS 1 6 1
RS 2 0 1 [1 0] 0
RS 1 5 1
RS 2 0 1 [1 0] 0
RS 1 4 1
RS 2 0 1 [1 0] 0
RS 1 3 1
RS 2 0 1 [1 0] 0
RS 1 2 1
RS 2 0 1 [1 0] 0
RS 1 1 1
RS 2 0 1 [1 0] 0
RS 1 0 1
RS 1 0 0 [1 0] 0
RS 1 0 0
RS 1 0 0 [1 0] 0
RS 1 1 0
RS 1 0 0 [1 0] 0
RS 1 2 0
RS 1 0 0 [1 0] 0
RS 1 3 0
RS 1 0 0 [1 0] 0
RS 1 4 0
RS 1 0 0 [1 0] 0
RS 1 5 0
RS 1 0 0 [1 0] 0
RS 1 6 0
RS 1 0 0 [1 0] 0
RS 1 7 0
RS 2 0 1 [1 0] 0
RS 1 0 1
RS 2 0 1 [1 0] 0
RS 1 1 1
RS 2 0 1 [1 0] 0
RS 1 2 1
RS 2 0 1 [1 0] 0
RS 1 3 1
RS 2 0 1 [1 0] 0
RS 1 4 1
RS 2 0 1 [1 0] 0
RS 1 5 1
RS 2 0 1 [1 0] 0
RS 1 6 1
RS 2 0 1 [1 0] 0
RS 1 7 1
RS 1 -1 0 [ 1 -1] 7
RS 0 4 49
RS 1 0 0 [1 0] 0
RS 0 3 0
RS 5 5 4 [1 1] 1
RS 0 2 11
RS 4 4 3 [1 1] 1
RS 0 2 10
RS 3 3 2 [1 1] 1
RS 0 2 9
RS 2 2 1 [1 1] 1
RS 0 2 8
RS 1 1

array([[ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1]], dtype=uint32)

In [34]:
eb.shape

(8, 8, 112)

In [36]:
ed.shape

(4672,)

In [10]:
mi

[0,
 1,
 136,
 129,
 511,
 911,
 904,
 897,
 1436,
 1437,
 1562,
 1561,
 1554,
 1575,
 1533,
 1540,
 1534,
 2665,
 2664,
 2663,
 2649,
 2677,
 2635,
 2636,
 1613,
 1728,
 1759,
 2051,
 730,
 1752,
 731]

In [11]:
b, r = replay(trace, 35)

In [12]:
expected_moves = list(b.legal_moves)

In [6]:
import sys
sys.path.append("/home2/jiasen/workspace/smart-chess/")

In [7]:
import chess
import libencoder
import encode
import utils

In [13]:
[(a == b, a, b) for a, b in zip([encode.MoveEncoding.decode(chess.BLACK, c)[0] for c in mi], expected_moves)]

[(True, Move.from_uci('h8h7'), Move.from_uci('h8h7')),
 (True, Move.from_uci('h8h6'), Move.from_uci('h8h6')),
 (True, Move.from_uci('g8h6'), Move.from_uci('g8h6')),
 (True, Move.from_uci('g8f6'), Move.from_uci('g8f6')),
 (True, Move.from_uci('a8a7'), Move.from_uci('a8a7')),
 (True, Move.from_uci('d7e8'), Move.from_uci('d7e8')),
 (True, Move.from_uci('d7d8'), Move.from_uci('d7d8')),
 (True, Move.from_uci('d7c8'), Move.from_uci('d7c8')),
 (True, Move.from_uci('e6f5'), Move.from_uci('e6f5')),
 (True, Move.from_uci('e6g4'), Move.from_uci('e6g4')),
 (True, Move.from_uci('c6c8'), Move.from_uci('c6c8')),
 (True, Move.from_uci('c6c7'), Move.from_uci('c6c7')),
 (True, Move.from_uci('c6b7'), Move.from_uci('c6b7')),
 (True, Move.from_uci('c6d6'), Move.from_uci('c6d6')),
 (True, Move.from_uci('c6c5'), Move.from_uci('c6c5')),
 (True, Move.from_uci('c6b5'), Move.from_uci('c6b5')),
 (True, Move.from_uci('c6c4'), Move.from_uci('c6c4')),
 (True, Move.from_uci('d4g7'), Move.from_uci('d4g7')),
 (True, Mo

In [3]:
m = chess.Move.from_uci("e6f5")
encode.MoveEncoding.encode(chess.BLACK, m), libencoder.encode_move(chess.BLACK, m)

PY 1 -1 0 <object type:list(int64)<iv=None>> 7
PY 2 3 49
RS 1 -1 0 [ 1 -1] 6
RS 2 3 42


(1436, 1429)

In [4]:
mr = utils.rotate(m)
encode.MoveEncoding.encode(chess.WHITE, mr), libencoder.encode_move(chess.WHITE, mr)

PY 0 <object type:UniTuple(int64 x 2)> 7
PY 2 3 49
RS 0 [ 1 -1] 6
RS 2 3 42


(1436, 1429)

In [5]:
encode.MoveEncoding.encode(chess.WHITE, chess.Move.from_uci("e6f5")), libencoder.encode_move(chess.WHITE, chess.Move.from_uci("e6f5"))

true, true, 1, 0


(3233, 3233)

In [6]:
mr

Move.from_uci('d3c4')

In [19]:
import queenmoves
import knightmoves
import underpromotions

In [20]:
queenmoves.encode(mr)

1436